In [ ]:
from experiments.attn.victory import *

In [ ]:
results = {}
for boardsize in (3, 5, 7, 9):
    for depth in [1, 2, 4, 8, 16]:
        for width in [8, 16, 32, 64, 128]:
            print(boardsize, depth, width)
            results[boardsize, depth, width] = run_trial(boardsize=boardsize, n_layers=depth, D=width)

In [ ]:
from pathlib import Path
import pickle
Path('output/experiments-victory.pkl').write_bytes(pickle.dumps(results))

In [ ]:
pickle.loads(Path('output/experiments-victory.pkl').read_bytes())

In [ ]:
from pavlov import stats, runs, storage
Model = common.FCModel
run = runs.new_run('experiments-victory', model=Model.__name__)

In [ ]:
for boardsize in (3, 5, 7, 9):
    for depth in [1, 2, 4, 8, 16]:
        for width in [8, 16, 32, 64, 128]:
            storage.snapshot(run, {'losses': results[boardsize, depth, width].to_dict()}, boardsize=boardsize, depth=depth, width=width)

In [ ]:
from pavlov import stats, runs, storage, files

In [ ]:
snapshots = pd.DataFrame.from_dict(storage.snapshots(run), orient='index')
df = {}
for i, row in snapshots.iterrows():
    df[row.boardsize, row.depth, row.width] = storage.load_snapshot(run, i)['losses']
df = pd

In [ ]:
df.index.name = 'step'
df.columns.names = ('boardsize', 'depth', 'width')

In [ ]:
finals = df.iloc[-1].unstack(-1).T

In [ ]:
import seaborn as sns
sns.heatmap(finals.applymap(np.log10), square=True)

In [ ]:
finals = df.iloc[-1].apply(np.log10).reset_index().rename(columns={999: 'val'})

In [ ]:
from statsmodels.formula import api as smf 
m = smf.ols('val ~ boardsize + depth + width + 1', finals).fit()

In [ ]:
r = m.resid.copy()
r.index = df.iloc[-1].index 

In [ ]:
vlim = r.abs().max()
sns.heatmap(r.unstack(2).T, cbar=False, square=True, cmap='RdBu', vmin=-vlim, vmax=+vlim)

In [ ]:
m.summary()